In [5]:
import numpy as np
import pandas as pd
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

### Extract All The Links

In [6]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# start at this page
driver.get('https://dialnet.unirioja.es/tesis/fechalectura?inicio=14000')

In [7]:
%%time
absLinks=[]
max_pages = 0
while True:
    link1 = driver.find_elements_by_css_selector("span.titulo")
    for tags in link1:
        link2 = tags.find_elements_by_tag_name("a")
        for link3 in link2:
            absLinks.append(link3.get_attribute("href"))
    
    # extract links
    try:
        time.sleep(7)
        driver.find_element_by_xpath('//*[@id="pieDeListadoDeBusquedaDeAutores"]/ul/li[4]/a').click() 
        max_pages += 1
        if max_pages == 170:
            break
    # stop if no more pages available
    except NoSuchElementException:
        break

Wall time: 23min 48s


### Get Contents From Each Link

In [8]:
data = []
for link in absLinks:
    time.sleep(5)
    driver.get(link)

    try:
        contents = driver.find_elements_by_xpath('//*[@id="resumen"]')
        for content in contents:
            language = content.text

    except NoSuchElementException:
        break

    #collect the data
    lang= {'content': language}
    data.append(lang)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)


In [9]:
en_sp = pd.DataFrame(data)

In [10]:
len(en_sp)

216

### Data Cleaning

In [11]:
# remove \n 
en_sp['content'] = en_sp['content'].apply(lambda x: re.sub(r'\n', '', x))

# return only rows with 'Espanol' in the text (because it contains english translate too)
en_sp['Spanish'] = en_sp['content'].apply(lambda x: x if 'Español' in x else 0)

# remove rows with 0
en_sp = en_sp[en_sp['Spanish'] != 0]

# create two new variables: Spanish_text, English_text to separate the translations
en_sp['Spanish_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[0])
en_sp['English_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[-1])

# remove 'Galego' translation in the English_text variable
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: x.split('Galego')[0] if 'Galego' in x else x)

en_sp['Spanish_text'] = en_sp['Spanish_text'].apply(lambda x: x.split('Español')[-1])

# remove any text that contains 'Español' in the English_text
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: 0 if 'Español' in x else x)

# remove rows with 0
en_sp = en_sp[en_sp['English_text'] != 0]


In [12]:
# make sure Spanish_text and English_text do not have the same value
en_sp = en_sp[en_sp['Spanish_text'] != en_sp['English_text']]
len(en_sp)

24

In [13]:
# format dataframe
en_sp.reset_index(inplace=True)
en_sp.drop(['index', 'content', 'Spanish'], axis=1, inplace=True)

In [14]:
en_sp = en_sp[:30]

In [15]:
en_sp['Spanish_text'][12]

'RESUMEN Antecedentes En los programas de cribado con mamografía digital, la doble lectura mamográfica es la estrategia de elección al ser más sensible en la detección de lesiones. Algunas de estas lesiones no son agresivas y pueden considerarse falsos positivos o sobrediagnóstico. Por tanto, es relevante evaluar la efectividad y coste-efectividad de la doble lectura y caracterizar las lesiones benignas.Objetivos Evaluar la efectividad y coste-efectividad de la doble lectura mamográfica en el contexto de la mamografía digital y valorar el comportamiento de lesiones benignas detectadas en un programa de cribado.Métodos Se realizaron cuatro estudios con diferentes diseños metodológicos. Para determinar la efectividad y el coste-efectividad de la doble lectura, se realizó un estudio de coste-consecuencia, uno de coste-efectividad y una revisión sistemática. Para determinar si la expresión de receptores de estrógeno (RE), progesterona (RP) y Ki-67 en las lesiones benignas aumenta el riesgo

In [16]:
en_sp['English_text'][12]

'ABSTRACT Background In breast cancer screening programmes, double reading of digital mammograms is used since it can detect more lesions. Some of these lesions are not aggressive and may be considered false positives or overdiagnosis. Therefore, it is relevant to evaluate the effectiveness and cost-effectiveness of double reading and to characterize benign lesions.Objectives To evaluate the effectiveness and cost-effectiveness of double reading of digital mammograms and to characterize the benign breast lesions detected in a breast cancer screening programme.Methods Four studies with different methodological designs were performed. To determine the effectiveness and cost-effectiveness of double reading of digital mammograms, we performed: i) a cost-consequence study; ii) a cost-effectiveness study; and iii) a systematic review of observational studies and economic evaluations. To determine whether the expression of estrogen (ER), progesterone (PR) and Ki-67 receptors in benign lesions

In [23]:
en_sp.to_csv('lang_doc4.csv', index=False)